Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# doing the import for numpy
import numpy as np


In [170]:
# Looking at the head of the dataFrame
print(df.shape)
df.head(5)

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [171]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
BOROUGH                           23040 non-null object
NEIGHBORHOOD                      23040 non-null object
BUILDING_CLASS_CATEGORY           23040 non-null object
TAX_CLASS_AT_PRESENT              23039 non-null object
BLOCK                             23040 non-null int64
LOT                               23040 non-null int64
EASE-MENT                         0 non-null float64
BUILDING_CLASS_AT_PRESENT         23039 non-null object
ADDRESS                           23040 non-null object
APARTMENT_NUMBER                  5201 non-null object
ZIP_CODE                          23039 non-null float64
RESIDENTIAL_UNITS                 23039 non-null float64
COMMERCIAL_UNITS                  23039 non-null float64
TOTAL_UNITS                       23039 non-null float64
LAND_SQUARE_FEET                  22987 non-null object
GROSS_SQUARE_FEET                 23039 non-null floa

In [0]:
# Will be dropping the Apartment number colum
df = df.drop(['APARTMENT_NUMBER', 'EASE-MENT'], axis=1)

In [175]:
df.shape

(23040, 19)

In [0]:
# I will drop the null values in the Land square feet and year built and others
df = df.dropna()

In [177]:
df.shape

(22953, 19)

In [204]:
theArry = df['LAND_SQUARE_FEET'].unique()
print(theArry)

['10,733' '2,962' '2,074' ... '3,914' '3,371' '5,036']


In [0]:
# writing a function that can then be applied to remove change the string to a integer
def changeStr(theStr):
  theStr = theStr.strip()
  return int(theStr.replace(",", ""))

In [206]:
# I am going to change the Land_square_feet to a number from a string
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].apply(changeStr)

ValueError: ignored

In [0]:
df['SALE_PRICE'].value_counts()

In [0]:
# Creating the subset of the dataFrame to have only single family dwellings
df = df[df["BUILDING_CLASS_CATEGORY"] == "01 ONE FAMILY DWELLINGS"]
df.shape

In [0]:
# Filtering to just the sale prices to a 
# sale price that was more than 100 thousand and less than 2 million.
df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000) ]
df.shape

In [0]:
# Do train/test split. Use data from January — March 2019 to train. 
# Use data from April 2019 to test.

# First will check to see if the sale_date is in DateTime format
print(df['SALE_DATE'].dtype)
df['SALE_DATE'].value_counts()

In [0]:
# Will change sale date to datetime format
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
myCutOff = pd.to_datetime('2019-03-31')
train = df[df['SALE_DATE'] <= myCutOff]
test = df[df['SALE_DATE'] > myCutOff]
print(train.shape, test.shape)
train['SALE_DATE'].value_counts()

In [0]:
# Looking to see what to do the one hot coding on
nonNumbers = train.describe(exclude=np.number)
nonNumbers

In [0]:
# Picking out the features that I feel I will want to drop.
# These will contain the high cardinal features and those that may have NAN values
features_to_drop = ['APARTMENT_NUMBER', 'ADDRESS', "EASE-MENT"]

In [0]:
# I will change the Land_square_feet to a number from a string


In [0]:
# Looking at the describe function for numerical values
train.describe().T

In [0]:
train['RESIDENTIAL_UNITS'].value_counts()

In [0]:
train['COMMERCIAL_UNITS'].value_counts()

In [0]:
train['TOTAL_UNITS'].value_counts()

In [0]:
train['BOROUGH'].value_counts(dropna=False)

In [0]:
train['NEIGHBORHOOD'].value_counts()  

In [0]:
train['APARTMENT_NUMBER'].value_counts() # Will drop this feature because mostly is NAN with only one value

In [0]:

train['TAX_CLASS_AT_PRESENT'].value_counts()

In [0]:
train['BUILDING_CLASS_AT_PRESENT'].value_counts()

In [0]:
train['BUILDING_CLASS_AT_TIME_OF_SALE'].value_counts()

In [0]:
train['BLOCK'].value_counts().sum()

In [0]:
train["LOT"].value_counts().sum()

In [0]:
train['LAND_SQUARE_FEET'].dtype
train['LAND_SQUARE_FEET'].value_counts().sum()

In [0]:
# Creating a function that will only keep those features that
# have a specified number of unique values
def getHighCardinalFeatures(dataFrame, numUnique):
  ''' 
    dataFrame:  the dataFrame passed in to drop features that have too many unique values
                this method is only checking those features that are categorical

    numUnique:  the max number of unique values that the feature can have to be kept

    
    Returns:  Will return a list of the features that to keep
  '''
  theColumns = dataFrame.columns
  
  # new list of the columns
  keep = []
  dropList = []

  
  
  for col in theColumns:
    if dataFrame[col].nunique() > numUnique :
      dropList.append()
      
  return  dropList

In [0]:
# Will drop the features that have high cardinality
train.head(3)

In [0]:
high_cardinality = getHighCardinalFeatures(nonNumbers, 20)
high_cardinality

In [0]:
#